<a href="https://colab.research.google.com/github/mongoq/thesis/blob/main/Classic_Motorway_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://towardsdatascience.com/implementing-real-time-object-detection-system-using-pytorch-and-opencv-70bac41148f7

In [8]:
! pip install pafy youtube-dl==2020.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import cv2
import pafy
import youtube_dl
import torch
from time import time

In [37]:
from torch import hub # Hub contains other models like FasterRCNN
#modeltorch = torch.hub.load( \
#                      'ultralytics/yolov5', \
#                      'yolov5s', \
#                      pretrained=True)
#print(type(modeltorch))

In [48]:
class Detect():

      #  Read this for classs magic:
      #  https://cito.github.io/byte_of_python/read/class-and-object-vars.html

      # init method or constructor
      def __init__(self):
        #from torch import hub # Hub contains other models like FasterRCNN
        self.modeltorch = torch.hub.load( \
                      'ultralytics/yolov5', \
                      'yolov5s', \
                      pretrained=True)
        print("init modeltorch: " + str(type(modeltorch)))
        #pass

      def get_video_stream(self):
        URL = "https://www.youtube.com/watch?v=wqctLW0Hb_0" #URL to parse
        play = pafy.new(URL).streams[-1] #'-1' means read the lowest quality of video.
        assert play is not None # we want to make sure their is a input to read.
        stream = cv2.VideoCapture(play.url) #create a opencv video stream.
        return stream

        """
        The function below identifies the device which is availabe to make the prediction and uses it to load and infer the frame. Once it has results it will extract the labels and cordinates(Along with scores) for each object detected in the frame.
        """
      #def score_frame(frame, model):
      def score_frame(self, frame):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("score_frame modeltorch: " + str(type(modeltorch)))
        self.modeltorch.to(device)
        frame = [torch.tensor(frame)]
        #frame = [int(torch.tensor(frame)).long()]
        #print("score_frame frame " + str(frame))
        results = self.modeltorch(frame)
        labels = results.xyxyn[0][:, -1].numpy()
        cord = results.xyxyn[0][:, :-1].numpy()
        return labels, cord

        """
        The function below takes the results and the frame as input and plots boxes over all the objects which have a score higer than our threshold.
        """
      def plot_boxes(self, results, frame):
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
          row = cord[i]
          # If score is less than 0.2 we avoid making a prediction.
          if row[4] < 0.2: 
            continue
          x1 = int(row[0]*x_shape)
          y1 = int(row[1]*y_shape)
          x2 = int(row[2]*x_shape)
          y2 = int(row[3]*y_shape)
          bgr = (0, 255, 0) # color of the box
          classes = self.modeltorch.names # Get the name of label index
          label_font = cv2.FONT_HERSHEY_SIMPLEX #Font for the label.
          cv2.rectangle(frame, \
                      (x1, y1), (x2, y2), \
                       bgr, 2) #Plot the boxes
          cv2.putText(frame,\
                    classes[labels[i]], \
                    (x1, y1), \
                    label_font, 0.9, bgr, 2) #Put a label over box.
          return frame

        """
        The Function below oracestrates the entire operation and performs the real-time parsing for video stream.
        """
      def __call__(self):
        out_file='motorway.mp4'
        player = self.get_video_stream() #Get your video stream.
        assert player.isOpened() # Make sure that their is a stream. 
        #Below code creates a new video writer object to write our
        #output stream.
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG") #Using MJPEG codex
        out = cv2.VideoWriter(out_file, four_cc, 20, \
                          (x_shape, y_shape)) 
        ret, frame = player.read() # Read the first frame.
        while ret: # Run until stream is out of frames
          start_time = time() # We would like to measure the FPS.
          results = self.score_frame(frame) # Score the Frame
          frame = self.plot_boxes(results, frame) # Plot the boxes.
          end_time = time()
          fps = 1/np.round(end_time - start_time, 3) #Measure the FPS.
          print(f"Frames Per Second : {fps}")
          out.write(frame) # Write the frame onto the output.
          ret, frame = player.read() # Read next frame.

In [49]:
Motorway=Detect()
#Motorway()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-9-20 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


init modeltorch: <class 'models.common.AutoShape'>


In [50]:
Motorway()

score_frame modeltorch: <class 'models.common.AutoShape'>


error: ignored

In [ ]:
print(type(Detect))

<class 'type'>
